In [1]:
import numpy as np

from automata.automata_rules import automata_output_list
from automata.schema_search_tools import (
    annihilation_generation_rules,
    fill_missing_outputs_as_maintenance,
    collect_data_from_generator,
    concatenate)
from cana.boolean_node import BooleanNode

# Crossover mechanism


In [2]:
# Define the crossover operation
from typing import List, Tuple


def crossover(
    parent1: str, parent2: str, prob: float = 0.5
) -> Tuple[List[str], List[str]]:
    """
    Crossover is a genetic operator used to combine the genetic information of two parents to produce new offspring.
    Its process is as follows:
    1. It splits the parent rules into annihilation and generation rules.
    2. Then it uses the function switcheroo() to crisscross each line of the rule over a randomly assigned crossover point. It does this with a probability {prob} for each line.
    3. Then it combines the annihilation and generation rules to form the children rules.
    4. It then creates a BooleanNode from the children rules using the function from_partial_lut().
    5. Finally, it fills the missing outputs as maintenance using the function fill_missing_outputs_as_maintenance().

    Parameters:
    parent1: list of output values
    parent2: list of output values
    prob: float, crossover probability

    Returns:
    child1: BooleanNode
    child2: BooleanNode
    """
    parent1 = [letter for letter in parent1]
    parent2 = [letter for letter in parent2]

    parent1_anni, parent1_gen = annihilation_generation_rules(parent1, split=True)
    parent2_anni, parent2_gen = annihilation_generation_rules(parent2, split=True)
    prob = 0.5
    child1_anni, child2_anni = switcheroo(
        parent1_anni, parent2_anni, prob=prob, type="annihilation"
    )
    child1_gen, child2_gen = switcheroo(
        parent1_gen, parent2_gen, prob=prob, type="generation"
    )
    child1 = child1_anni + child1_gen
    child1 = BooleanNode.from_partial_lut(child1, fill_clashes=True)
    child1 = fill_missing_outputs_as_maintenance(child1)

    child2 = child2_anni + child2_gen
    child2 = BooleanNode.from_partial_lut(child2, fill_clashes=True)
    child2 = fill_missing_outputs_as_maintenance(child2)

    return child1.outputs, child2.outputs


def switcheroo(parent1: list, parent2: list, prob: float=0.5, type: str=None)-> list:
    """
    Switcheroo is a crossover helper. After the parent rules are split into annihilation and generation, they are fed to switcheroo.
    Its process is as follows:
    1. It extracts only the inputs from the rules.
    2. It checks if the input length is odd and raises an error if it is not. 
    3. It takes out the middle element of the rule inputs (to ensure that the annihilation or generation aspect is preserved when switching).
    4. It then selects a random crossover point from the length of the first input.
    5. Then with a probability prob, it crisscrosses the rules at the crossover point between the parents.
    6. Finally, it adds the middle element back to the rules, attaches the output values to the rules and returns the children as a list of schemata rules.


    Parameters:
    parent1: list of schemata rules
    parent2: list of schemata rules
    prob: float, crossover probability
    type: str, type of rules that are being switched. Either 'generation' or 'annihilation'

    Returns:
    child1: list of schemata rules
    child2: list of schemata rules
    """
    if type not in ["generation", "annihilation"]:
        raise ValueError("type should be either 'generation' or 'annihilation'")

    # extracting only inputs
    parent1 = [rule[0] for rule in parent1]
    parent2 = [rule[0] for rule in parent2]

    k = len(parent1[0])
    # print(f"Size of the neighbourhood: {k}")
    # check if the neighbourhood size is odd. return error if even
    if k < 3:
        raise ValueError("Neighbourhood size should be at least 3")
    if k % 2 == 0:
        raise ValueError("Neighbourhood size should be odd")

    # removing middle element to preserve annihilations and generations, assuming the total neighborhood size is k
    parent1 = [rule[: k // 2] + rule[((k // 2) + 1) :] for rule in parent1]
    parent2 = [rule[: k // 2] + rule[((k // 2) + 1) :] for rule in parent2]
    # print(f"Parent 1: {parent1}")
    # print(f"Parent 2: {parent2}")

    # crossover
    index = np.random.randint(
        0, k
    )  # taking random crossover point from the length of the first input
    # print(f"Crossover Index: {index}")
    child1 = []
    child2 = []

    for i in range(len(parent1)):
        for j in range(len(parent2)):
            if np.random.rand() < prob:
                child1.append(parent1[i][:index] + parent2[j][index:])
                child2.append(parent2[j][:index] + parent1[i][index:])
    if len(child1) == 0:
        child1 = parent1
    if len(child2) == 0:
        child2 = parent2

    # adding middle element back
    if type == "generation":
        filler = "0"
        not_filler = "1"
    elif type == "annihilation":
        filler = "1"
        not_filler = "0"

    child1 = [[rule[:3] + filler + rule[3:], not_filler] for rule in child1]
    child2 = [[rule[:3] + filler + rule[3:], not_filler] for rule in child2]


    return child1, child2

In [5]:
rule1 = "MM401"
rule2 = "GP"
parent1 =concatenate(automata_output_list[rule1])
print(f"anni_gen {rule1}:\n{annihilation_generation_rules(parent1)}")
parent2 = concatenate(automata_output_list[rule2])
print(f"anni_gen {rule2}:\n{annihilation_generation_rules(parent2)}")

child1, child2 = crossover(parent1, parent2)
print(f"child1:\n{child1}")
print(f"child2:\n{child2}")
anni_gen_child1 = annihilation_generation_rules(child1)
anni_gen_child2 = annihilation_generation_rules(child2)
print(f"anni_gen child1:\n{anni_gen_child1}")
print(f"anni_gen child2:\n{anni_gen_child2}")


anni_gen MM401:
[['##1111#', 0], ['###1##1', 0], ['0##0###', 1], ['#0000##', 1]]
anni_gen GP:
[['0##10##', 0], ['0#01###', 0], ['0##1##0', 0], ['##10##1', 1], ['1##0##1', 1], ['###01#1', 1]]
child1:
['0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '1']
child2:
['1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '0', '